In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
from pathlib import Path
import json

import numpy as np
import pandas as pd
import anndata as ad

import matplotlib.pyplot as plt

%matplotlib inline

In [5]:
a = ['x_reconstructed','y_reconstructed',
                                        'z_reconstructed']
len(a)

In [9]:
ABC_ROOT = Path("/data/abc_atlas/")

In [14]:
ccf_df = pd.read_csv(
            ABC_ROOT/"metadata/Allen-CCF-2020/20230630/parcellation_to_parcellation_term_membership.csv"
            )
ccf_df
# [(ccf_df['parcellation_term_acronym']=='TH')
       # |(ccf_df['parcellation_term_acronym']=='ZI')]

In [11]:
ccf_set_df = pd.read_csv(
            ABC_ROOT/"metadata/Allen-CCF-2020/20230630/parcellation_term_set_membership.csv"
            )
ccf_set_df

# Load abc atlas metadata

In [2]:
abc_root = Path("/data/abc_atlas/")
brain_id = 'C57BL6J-638850'
version = '20230830'

In [3]:
# load in the 'obs' cluster_annotations + parcellation_annotation metadata
ccf_md_df = pd.read_csv(abc_root/f'metadata/MERFISH-{brain_id}-CCF/{version}/views/cell_metadata_with_parcellation_annotation.csv', 
                        dtype={'cell_label':str}, 
                        index_col=0)

# flip y coords so the sections don't display upside down
flip_y=True
if flip_y:
    ccf_md_df[['y_section', 'y_reconstructed']] *= -1

# Define an expansive spatial boundary of thalamus

In [118]:
from shapely import Polygon, MultiPoint, LineString
from shapely.plotting import plot_polygon, plot_line
import shapely
import ccf_polygons as cpoly

from importlib import reload
reload(cpoly);

In [119]:
# find the sections that contain TH or ZI cells
th_zi_sections = ccf_md_df[(ccf_md_df['parcellation_division']=='TH') 
                           | (ccf_md_df['parcellation_structure']=='ZI')
                          ]['brain_section_label'].unique()
th_zi_sections = sorted(list(th_zi_sections))
# len(th_zi_sections)

In [129]:
test_sec = th_zi_sections[6]
th_zi_df = ccf_md_df[( (ccf_md_df['parcellation_division']=='TH') 
                       | (ccf_md_df['parcellation_structure']=='ZI'))
                     & (ccf_md_df['brain_section_label']==test_sec)]

test_poly = cpoly.get_polygon_from_obs(th_zi_df, strategy='midline',
                                        x_field='x_section', y_field='y_section')
test_poly

In [121]:
test_poly

plt.scatter(th_zi_df.x_section, th_zi_df.y_section, s=0.1, c='lightgrey')
plt.gca().axis('equal')
plot_polygon(test_poly, add_points=False, facecolor='none', color='b')
plt.show()

In [130]:
th_zi_convex_hull_dict = dict()
th_zi_concave_hull_dict = dict()
th_zi_concave_hull_hdbscan_dict = dict()
th_zi_concave_hull_midline_dict = dict()
th_zi_concave_hull_hdbscan_buffer_dict = dict()
th_zi_concave_hull_midline_buffer_dict = dict()

for i, sec in enumerate(th_zi_sections):
    curr_sec_id = sec

    th_zi_df = ccf_md_df[( (ccf_md_df['parcellation_division']=='TH') 
                           | (ccf_md_df['parcellation_structure']=='ZI'))
                         & (ccf_md_df['brain_section_label']==curr_sec_id)].copy()
    if th_zi_df.empty:
        continue

    # make a shapely multipoint object
    th_zi_coords = list(zip(th_zi_df.x_section, th_zi_df.y_section))
    th_zi_multipoint = MultiPoint(th_zi_coords)
    # th_zi_multipoint.bounds

    # set buffer distance
    buffer_dist = 0.1

    # calculate convex hull
    th_zi_convex_hull = th_zi_multipoint.convex_hull
    # use buffer to expand convex hull outline a bit
    th_zi_convex_hull_buffer = shapely.buffer(th_zi_convex_hull, buffer_dist)
    # store
    th_zi_convex_hull_dict[curr_sec_id] = th_zi_convex_hull_buffer

    # calculate concave hull
    # standard on all cells
    th_zi_concave_hull = shapely.concave_hull(th_zi_multipoint)
    # use buffer to expand to something that's more like an outline
    th_zi_concave_hull_buffer = shapely.buffer(th_zi_concave_hull, buffer_dist)
    # remove interior holes by making a new Polygon out of the exterior coords
    th_zi_concave_hull_buffer = Polygon(th_zi_concave_hull_buffer.exterior.coords)
    # store
    th_zi_concave_hull_dict[curr_sec_id] = th_zi_concave_hull_buffer
    
    # attempt to split cells across the midline before doing concave hull
    # using hdbscan method
    th_zi_concave_hull_hdbscan = cpoly.get_polygon_from_obs(th_zi_df, 
                                                            strategy='hdbscan',
                                                            x_field='x_section', 
                                                            y_field='y_section')
    th_zi_concave_hull_hdbscan_dict[curr_sec_id] = th_zi_concave_hull_hdbscan
     # use buffer to expand to something that's more like an outline
    th_zi_concave_hull_hdbscan_buffer = shapely.buffer(th_zi_concave_hull_hdbscan, buffer_dist)
    # remove interior holes by making a new Polygon out of the exterior coords
    if th_zi_concave_hull_hdbscan_buffer.geom_type == 'Polygon':
        th_zi_concave_hull_hdbscan_buffer = Polygon(th_zi_concave_hull_hdbscan_buffer.exterior.coords)
    # store
    th_zi_concave_hull_hdbscan_buffer_dict[curr_sec_id] = th_zi_concave_hull_hdbscan_buffer
    
    # using midline method
    th_zi_concave_hull_midline = cpoly.get_polygon_from_obs(th_zi_df, 
                                                            strategy='midline',
                                                            x_field='x_section', 
                                                            y_field='y_section')
    th_zi_concave_hull_midline_dict[curr_sec_id] = th_zi_concave_hull_midline
     # use buffer to expand to something that's more like an outline
    th_zi_concave_hull_midline_buffer = shapely.buffer(th_zi_concave_hull_midline, buffer_dist)
    # remove interior holes by making a new Polygon out of the exterior coords
    if th_zi_concave_hull_midline_buffer.geom_type == 'Polygon':
        th_zi_concave_hull_midline_buffer = Polygon(th_zi_concave_hull_midline_buffer.exterior.coords)
    # store
    th_zi_concave_hull_midline_buffer_dict[curr_sec_id] = th_zi_concave_hull_midline_buffer

In [131]:
th_zi_concave_hull_dict

In [136]:
n_col = 3
n_row = int(np.ceil(len(th_zi_sections) / n_col))
fig, axes = plt.subplots(n_row, n_col, figsize=(16,16))
axes = axes.ravel()

x_min = 2.5
x_max = 8.5
y_min = -7.5
y_max = -3.5

for i, sec in enumerate(th_zi_sections):
    ax = axes[i]
    
    curr_sec_df = ccf_md_df[( (ccf_md_df['parcellation_division']=='TH') 
                              | (ccf_md_df['parcellation_structure']=='ZI'))
                            & (ccf_md_df['brain_section_label']==sec)
                            & (ccf_md_df['x_section']>x_min)
                            & (ccf_md_df['x_section']<x_max)
                            & (ccf_md_df['y_section']>y_min)
                            & (ccf_md_df['y_section']<y_max)]
    
    ax.scatter(curr_sec_df.x_section, curr_sec_df.y_section, s=0.1, c='lightgrey')
    ax.axis('equal')
    ax.axis('off')
    ax.set_xlim((2,9))
    ax.set_ylim((-8,-3))
    ax.set_xticks([])
    ax.set_yticks([])
    # plot_polygon(th_zi_concave_hull_dict[sec], ax=ax, 
    #              add_points=False, facecolor='none', color='k', label='concave hull w/ buffer')
    plot_polygon(th_zi_concave_hull_midline_dict[sec], ax=ax, 
                 add_points=False, facecolor='none', color='magenta', label='concave hull w/ midline split')
    plot_polygon(th_zi_concave_hull_midline_buffer_dict[sec], ax=ax, 
                 add_points=False, facecolor='none', color='r', label='concave hull w/ midline split & buffer')
    plot_polygon(th_zi_concave_hull_hdbscan_dict[sec], ax=ax, 
                 add_points=False, facecolor='none', color='deepskyblue', label='concave hull w/ hdbscan split')
    plot_polygon(th_zi_concave_hull_hdbscan_buffer_dict[sec], ax=ax, 
                 add_points=False, facecolor='none', color='b', label='concave hull w/ hdbscan split, exterior only, & buffer')
    if i==0:
        ax.legend(loc='lower left', bbox_to_anchor=(0.1,1.1))

In [137]:
n_col = 3
n_row = int(np.ceil(len(th_zi_sections) / n_col))
fig, axes = plt.subplots(n_row, n_col, figsize=(18,24))
axes = axes.ravel()

x_min = 2.5
x_max = 8.5
y_min = -7.5
y_max = -3.5

for i, sec in enumerate(th_zi_sections):
    ax = axes[i]
    
    curr_sec_df = ccf_md_df[(ccf_md_df['brain_section_label']==sec)
                            & (ccf_md_df['x_section']>x_min)
                            & (ccf_md_df['x_section']<x_max)
                            & (ccf_md_df['y_section']>y_min)
                            & (ccf_md_df['y_section']<y_max)]

    ax.scatter(curr_sec_df['x_section'], curr_sec_df['y_section'],
                color=curr_sec_df['parcellation_substructure_color'],
                s=0.5, marker='.', zorder=1)
    
    ax.set_xlim((x_min,x_max))
    ax.set_ylim((y_min,y_max))
    ax.axis('equal')
    ax.axis('off')
    ax.set_xticks([])
    ax.set_yticks([])
    
    plot_polygon(th_zi_concave_hull_midline_buffer_dict[sec], ax=ax, 
                 add_points=False, facecolor='none', color='k', 
                 label='concave hull w/ midline split & buffer added')
    if i==0:
        ax.legend(loc='lower left', bbox_to_anchor=(0.1,1.1)) 

In [2]:
n_col = 3
n_row = int(np.ceil(len(th_zi_sections) / n_col))
fig, axes = plt.subplots(n_row, n_col, figsize=(18,24))
axes = axes.ravel()

x_min = 2.5
x_max = 8.5
y_min = -7.5
y_max = -3.5

for i, sec in enumerate(th_zi_sections):
    ax = axes[i]
    
    curr_sec_df = ccf_md_df[(ccf_md_df['brain_section_label']==sec)
                            & (~ccf_md_df['neurotransmitter'].isna())
                            & (ccf_md_df['x_section']>x_min)
                            & (ccf_md_df['x_section']<x_max)
                            & (ccf_md_df['y_section']>y_min)
                            & (ccf_md_df['y_section']<y_max)]

    ax.scatter(curr_sec_df['x_section'], curr_sec_df['y_section'],
                color=curr_sec_df['class_color'],
                s=0.5, marker='.', zorder=1)
    
    ax.set_xlim((x_min,x_max))
    ax.set_ylim((y_min,y_max))
    ax.axis('equal')
    ax.axis('off')
    ax.set_xticks([])
    ax.set_yticks([])
    
    plot_polygon(th_zi_concave_hull_midline_buffer_dict[sec], ax=ax, lw=2,
                 add_points=False, facecolor='none', color='k',
                 label='concave hull w/ midline split & buffer added')
    if i==0:
        ax.legend(loc='lower left', bbox_to_anchor=(0.1,1.1))